In [189]:
reset -fs

In [190]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model, tree
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
 
from sklearn.metrics import roc_curve, confusion_matrix,precision_recall_curve,roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score 

import xgboost as xgb # note for installation use py-xgboost in conda

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
plt.style.use('seaborn')
np.random.seed(42)

In [191]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('application_test.csv')
test['target'] = -999
train.columns = train.columns.str.lower()
test.columns = test.columns.str.lower()

train['dataset'], test['dataset'] = 0, 1

X = pd.concat([train,test],sort=True)

In [192]:
X['age'] = X['days_birth'].apply(lambda x: int(x / -365))
X.drop('days_birth', axis=1, inplace=True)

In [193]:
X['employed'] = X['days_employed'].apply(lambda x: int(x / -365))
X.drop('days_employed', axis=1, inplace=True)
X = X[X['employed'] != -1000]

In [194]:
X = X[['target', 'dataset', 'age', 'employed', 'code_gender',
       'ext_source_1', 'ext_source_2','ext_source_3',
      ]]

In [195]:
num_features = list(X.columns[X.dtypes!=object])
num_features.remove('target')
cat_features = list(X.columns[X.dtypes==object])

In [196]:
for i in X.columns:
    X[i] = X[i].apply(lambda x: float('NaN') if x == 'XNA' else x)

In [197]:
X[cat_features] = X[cat_features].fillna('missing')

In [198]:
X = X.join(pd.get_dummies(X[cat_features], drop_first=True))
X.drop(cat_features, axis=1, inplace=True)

In [199]:
X[num_features] = X[num_features].fillna(X[num_features].median())

In [201]:
X_train = X[X.dataset == 0].drop(['target', 'dataset'], axis=1)
y_train = X[X.dataset == 0].target

In [202]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

model = DecisionTreeClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1, verbose=5)
print('Mean ROC AUC: %.3f' % np.mean(scores))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   11.3s


Mean ROC AUC: 0.608


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   39.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   39.4s finished
